<a href="https://colab.research.google.com/github/AlekhyaGangopadhyay/IITPatna_Internship_LLMs/blob/main/DeepSeek_VL2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#  DeepSeek-VL2  ▸  Finance Harassment/Abuse Video Dataset
#  Full Pipeline : Train → Infer → Accuracy Score
#  Dataset       : iamalekhya/Finance_set  (318 videos, 9 cols)
#  Run on        : Google Colab → Runtime ▸ T4 GPU
#
#  Model variants & VRAM needed:
#    deepseek-vl2-tiny  →  ~14 GB  ✅ T4 (16 GB)   ← USE THIS
#    deepseek-vl2-small →  ~40 GB  ⚠️  A100 only
#    deepseek-vl2       →  ~80 GB  ❌  multi-GPU
# ============================================================

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 1 ── Install  (run once → Runtime ▸ Restart session)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# DeepSeek-VL2 requires its own package + pinned transformers
!pip install git+https://github.com/deepseek-ai/DeepSeek-VL2.git --no-deps
!pip install attrdict timm "transformers<4.48.0"
!pip install peft accelerate bitsandbytes rouge-score openpyxl opencv-python scikit-learn
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 2 ── Mount Google Drive
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from google.colab import drive
drive.mount('/content/drive')

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 3 ── ⚙️  CONFIG  ← only edit this section
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Model — use tiny for T4, small for A100
MODEL_PATH  = "deepseek-ai/deepseek-vl2-tiny"

# Dataset
HF_DATASET     = "iamalekhya/Finance_set"
EXCEL_FALLBACK = "/content/drive/MyDrive/IITP_Internship/Finance_set.xlsx"

# Videos in Google Drive (searches all folders)
VIDEO_DIRS = [
    "/content/drive/Shared drives/Finance_video_Anamoly/AG",
    "/content/drive/Shared drives/Finance_video_Anamoly/ag",
    "/content/drive/Shared drives/Finance_video_Anamoly/Alekhya",
    "/content/drive/Shared drives/Finance_video_Anamoly/Pritam",
    "/content/drive/Shared drives/Finance_video_Anamoly/Videos",
]
VIDEO_EXT = ".mp4"

# Training
NUM_FRAMES  = 8       # frames extracted per video (keep ≤8 for T4)
MAX_SEQ_LEN = 1024    # lower = less VRAM
EPOCHS      = 3
BATCH_SIZE  = 1
GRAD_ACCUM  = 8
LORA_RANK   = 8       # lower than Qwen since MoE already efficient
EVAL_SPLIT  = 0.10
RANDOM_SEED = 42

# Output paths
OUTPUT_DIR          = "./deepseek_vl2_finance"
ADAPTER_DRIVE       = "/content/drive/MyDrive/IIT_Internship/deepseek_vl2_adapter"
RESULTS_EXCEL_DRIVE = "/content/drive/MyDrive/IIT_Internship/deepseek_finance_results.xlsx"

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 4 ── Imports
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import os, re, json, cv2, shutil, torch, warnings
import numpy as np
import pandas as pd
from PIL import Image
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from rouge_score import rouge_scorer as rs
from torch.utils.data import Dataset as TorchDataset
from torch.nn.utils.rnn import pad_sequence
from transformers import TrainingArguments, Trainer

warnings.filterwarnings("ignore")
print("✅ Imports OK")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 5 ── Load Dataset
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("Loading dataset...")
try:
    ds = load_dataset(HF_DATASET, split="train")
    df = ds.to_pandas()
    print(f"✅ Loaded from HuggingFace: {len(df)} rows")
except Exception as e:
    print(f"⚠️  HF failed ({e}), loading from Excel...")
    df = pd.read_excel(EXCEL_FALLBACK)
    print(f"✅ Loaded from Excel: {len(df)} rows")

df.columns = [c.strip() for c in df.columns]
REQUIRED = ["uni_id","Category","Aspect","Intent","Explanation","Country","Source","Sector"]
missing  = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

df = df.dropna(subset=REQUIRED).reset_index(drop=True)
print(f"   Rows: {len(df)} | Categories: {df['Category'].unique().tolist()}")

ALL_ASPECTS = sorted(df["Aspect"].unique().tolist())

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 6 ── Prompts  (DeepSeek-VL2 style)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# DeepSeek-VL2 uses <image> token per image in content string
# and roles must be exactly "<|User|>" and "<|Assistant|>"

def build_user_content(num_frames: int) -> str:
    """Build the user message string with <image> tokens for each frame."""
    image_tags = "\n".join([f"Frame {i+1}: <image>" for i in range(num_frames)])
    return f"""{image_tags}

You are an expert video analyst specialising in financial harassment, fraud, abuse, and workplace misconduct.

Analyse the video frames above and return a single valid JSON object with EXACTLY these fields:
{{
  "category"   : "Harassment" | "Bullying",
  "aspect"     : one of the known aspect labels listed below,
  "intent"     : 1 (intentional) | 0 (unintentional),
  "explanation": "1-2 sentence description of what is happening and why it is problematic",
  "country"    : "country name or Unknown",
  "source"     : "YouTube | News | Training | Documentary | Other",
  "sector"     : "Finance | Healthcare | Education | Retail | Tech | Government | General"
}}

Known aspect labels (pick the closest match):
{json.dumps(ALL_ASPECTS, indent=2)}

Rules:
- Output ONLY the JSON. No preamble, no markdown fences, no extra text.
- Use double quotes for all strings.
- intent must be an integer (1 or 0), not a string.

Carefully observe:
• Who is involved and what is happening
• The type and form of misconduct
• Whether the act is intentional
• The country/industry context"""

SYSTEM_PROMPT = "You are an expert video analyst specialising in financial harassment, fraud, abuse, and workplace misconduct. Always respond with valid JSON only."

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 7 ── Helpers
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
def find_video(uni_id: str) -> str:
    filename = f"{uni_id}{VIDEO_EXT}"
    for folder in VIDEO_DIRS:
        path = os.path.join(folder, filename)
        if os.path.exists(path):
            return path
        if os.path.isdir(folder):
            for f in os.listdir(folder):
                if f.lower() == filename.lower():
                    return os.path.join(folder, f)
    return ""


def extract_frames(video_path: str, n: int = NUM_FRAMES) -> list:
    if not video_path or not os.path.exists(video_path):
        return []
    cap   = cv2.VideoCapture(video_path)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total == 0:
        cap.release()
        return []
    frames = []
    for idx in np.linspace(0, total - 1, n, dtype=int):
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            frames.append(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    cap.release()
    return frames


def row_to_json(row) -> str:
    return json.dumps({
        "category"   : str(row["Category"]).strip(),
        "aspect"     : str(row["Aspect"]).strip(),
        "intent"     : int(row["Intent"]),
        "explanation": str(row["Explanation"]).strip(),
        "country"    : str(row["Country"]).strip(),
        "source"     : str(row["Source"]).strip(),
        "sector"     : str(row["Sector"]).strip(),
    }, indent=2)


def parse_json_response(text: str) -> dict:
    try:
        m = re.search(r'\{.*\}', text, re.DOTALL)
        if m:
            return json.loads(m.group())
    except Exception:
        pass
    return {}

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 8 ── Load Model + Processor + LoRA
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print(f"\nLoading {MODEL_PATH}...")

# DeepSeek-VL2 uses its own processor (not HuggingFace AutoProcessor)
vl_processor: DeepseekVLV2Processor = DeepseekVLV2Processor.from_pretrained(
    MODEL_PATH, trust_remote_code=True
)
tokenizer = vl_processor.tokenizer

# Load model in bfloat16 (4-bit not officially supported for DeepSeek-VL2)
vl_model: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    trust_remote_code = True,
    torch_dtype       = torch.bfloat16,
    device_map        = "auto",        # auto splits across available GPU(s)
)
vl_model.eval()

# Apply LoRA to language backbone only
# DeepSeek-VL2 language model is accessed via .language attribute
lora_config = LoraConfig(
    task_type        = TaskType.CAUSAL_LM,
    r                = LORA_RANK,
    lora_alpha       = LORA_RANK * 2,
    lora_dropout     = 0.05,
    target_modules   = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
    bias             = "none",
)
vl_model.language = get_peft_model(vl_model.language, lora_config)
vl_model.language.print_trainable_parameters()
print("✅ Model + LoRA ready")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 9 ── Build Samples
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
train_df, eval_df = train_test_split(df, test_size=EVAL_SPLIT, random_state=RANDOM_SEED)
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"\nSplit → Train: {len(train_df)}  |  Eval: {len(eval_df)}")


def build_samples(dataframe, split_name="train"):
    """
    Each sample stores:
      - pil_images : list of PIL frames
      - conversation: DeepSeek-VL2 conversation dict format
      - target_json : ground truth answer string
      - uni_id      : video id
    """
    samples, skipped = [], 0

    for i, row in dataframe.iterrows():
        uni_id     = str(row["uni_id"]).strip()
        video_path = find_video(uni_id)
        frames     = extract_frames(video_path)

        if not frames:
            print(f"  ⚠️  [{split_name}] Skipping {uni_id} — not found")
            skipped += 1
            continue

        n_frames    = len(frames)
        target      = row_to_json(row)
        user_text   = build_user_content(n_frames)

        # DeepSeek-VL2 conversation format
        conversation = [
            {
                "role"   : "<|User|>",
                "content": user_text,
                "images" : frames,      # list of PIL Images directly
            },
            {
                "role"   : "<|Assistant|>",
                "content": target,
            },
        ]

        samples.append({
            "conversation": conversation,
            "pil_images"  : frames,
            "target_json" : target,
            "uni_id"      : uni_id,
        })

        if (i + 1) % 30 == 0:
            print(f"  [{split_name}] {i+1}/{len(dataframe)}  skipped={skipped}")

    print(f"  ✅ [{split_name}] ready={len(samples)}  skipped={skipped}")
    return samples


print("\n📦 Building training samples...")
train_samples = build_samples(train_df, "train")

print("\n📦 Building eval samples...")
eval_samples  = build_samples(eval_df,  "eval")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 10 ── PyTorch Dataset + Collator
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
class DeepSeekVL2Dataset(TorchDataset):
    """
    Tokenises each sample using DeepseekVLV2Processor on-the-fly.
    The processor handles <image> token expansion and vision encoding.
    """
    def __init__(self, samples, processor, max_length=1024):
        self.samples    = samples
        self.processor  = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s            = self.samples[idx]
        conversation = s["conversation"]
        pil_images   = s["pil_images"]

        # vl_processor prepares input_ids, attention_mask, pixel_values etc.
        inputs = self.processor(
            conversations  = conversation,
            images         = pil_images,
            force_batchify = True,
            system_prompt  = SYSTEM_PROMPT,
        )

        input_ids = inputs.input_ids.squeeze(0)

        # Truncate if too long
        if input_ids.shape[0] > self.max_length:
            input_ids = input_ids[:self.max_length]

        # Labels: mask everything except the assistant response with -100
        # Find assistant response start by locating eos of user turn
        labels = input_ids.clone()
        # Find the separator between user and assistant in token ids
        # DeepSeek uses <|Assistant|> token; mask user portion
        assistant_token_id = self.processor.tokenizer.convert_tokens_to_ids("<|Assistant|>")
        assist_positions   = (input_ids == assistant_token_id).nonzero(as_tuple=True)[0]
        if len(assist_positions) > 0:
            first_assist = assist_positions[0].item()
            labels[:first_assist + 1] = -100   # mask user + assistant header

        # Mask pad tokens
        pad_id = self.processor.tokenizer.pad_token_id or 0
        labels[labels == pad_id] = -100

        item = {
            "input_ids"     : input_ids,
            "attention_mask": (input_ids != pad_id).long(),
            "labels"        : labels,
        }

        # Add pixel_values if present
        if hasattr(inputs, "pixel_values") and inputs.pixel_values is not None:
            item["pixel_values"] = inputs.pixel_values.squeeze(0)

        if hasattr(inputs, "images_seq_mask") and inputs.images_seq_mask is not None:
            item["images_seq_mask"] = inputs.images_seq_mask.squeeze(0)

        if hasattr(inputs, "images_emb_mask") and inputs.images_emb_mask is not None:
            item["images_emb_mask"] = inputs.images_emb_mask.squeeze(0)

        return item


def collate_fn(batch):
    pad_id = tokenizer.pad_token_id or 0

    input_ids      = pad_sequence([b["input_ids"]      for b in batch], batch_first=True, padding_value=pad_id)
    attention_mask = pad_sequence([b["attention_mask"] for b in batch], batch_first=True, padding_value=0)
    labels         = pad_sequence([b["labels"]         for b in batch], batch_first=True, padding_value=-100)

    result = {
        "input_ids"     : input_ids,
        "attention_mask": attention_mask,
        "labels"        : labels,
    }

    if "pixel_values" in batch[0]:
        try:
            result["pixel_values"] = torch.stack([b["pixel_values"] for b in batch])
        except Exception:
            pass  # skip if shapes differ (dynamic tiling)

    if "images_seq_mask" in batch[0]:
        result["images_seq_mask"] = pad_sequence(
            [b["images_seq_mask"] for b in batch], batch_first=True, padding_value=False
        )

    if "images_emb_mask" in batch[0]:
        result["images_emb_mask"] = pad_sequence(
            [b["images_emb_mask"] for b in batch], batch_first=True, padding_value=False
        )

    return result


train_torch_ds = DeepSeekVL2Dataset(train_samples, vl_processor, max_length=MAX_SEQ_LEN)
print(f"✅ PyTorch train dataset: {len(train_torch_ds)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 11 ── Train
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
vl_model.train()

training_args = TrainingArguments(
    output_dir                  = OUTPUT_DIR,
    per_device_train_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = GRAD_ACCUM,
    num_train_epochs            = EPOCHS,
    learning_rate               = 2e-4,
    bf16                        = True,
    logging_steps               = 5,
    save_steps                  = 100,
    save_total_limit            = 2,
    warmup_ratio                = 0.03,
    lr_scheduler_type           = "cosine",
    optim                       = "adamw_torch",
    seed                        = RANDOM_SEED,
    remove_unused_columns       = False,
    dataloader_pin_memory       = False,
    report_to                   = "none",
)

trainer = Trainer(
    model         = vl_model,
    args          = training_args,
    train_dataset = train_torch_ds,
    data_collator = collate_fn,
)

print("\n🚀 Training started...")
trainer.train()
print("✅ Training complete!")

# Save LoRA adapter
vl_model.language.save_pretrained("deepseek_vl2_lora_adapter")
tokenizer.save_pretrained("deepseek_vl2_lora_adapter")
os.makedirs(ADAPTER_DRIVE, exist_ok=True)
shutil.copytree("deepseek_vl2_lora_adapter", ADAPTER_DRIVE, dirs_exist_ok=True)
print(f"✅ LoRA adapter saved → {ADAPTER_DRIVE}")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 12 ── Inference Function
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
vl_model.eval()

def predict(frames: list) -> dict:
    """
    Run DeepSeek-VL2 inference on a list of PIL frames.
    Returns parsed JSON dict with predicted fields.
    """
    n_frames     = len(frames)
    user_content = build_user_content(n_frames)

    conversation = [
        {
            "role"   : "<|User|>",
            "content": user_content,
            "images" : frames,
        },
        {
            "role"   : "<|Assistant|>",
            "content": "",             # empty = model will fill this
        },
    ]

    # Prepare inputs using DeepSeek's processor
    prepare_inputs = vl_processor(
        conversations  = conversation,
        images         = frames,
        force_batchify = True,
        system_prompt  = SYSTEM_PROMPT,
    ).to(vl_model.device)

    # Get image embeddings + run generation
    with torch.no_grad():
        inputs_embeds = vl_model.prepare_inputs_embeds(**prepare_inputs)

        outputs = vl_model.language.generate(
            inputs_embeds  = inputs_embeds,
            attention_mask = prepare_inputs.attention_mask,
            pad_token_id   = tokenizer.eos_token_id,
            bos_token_id   = tokenizer.bos_token_id,
            eos_token_id   = tokenizer.eos_token_id,
            max_new_tokens = 512,
            temperature    = 0.1,
            do_sample      = False,
            use_cache      = True,
        )

    answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
    return parse_json_response(answer)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 13 ── Run Inference on ALL samples (train + eval)
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
print("\n" + "="*65)
print("  🔍 Running inference on ALL samples (train + eval)")
print("="*65)

all_samples = train_samples + eval_samples
all_results = []

for i, s in enumerate(all_samples):
    pred = predict(s["pil_images"])
    tgt  = json.loads(s["target_json"])

    all_results.append({
        "uni_id"          : s["uni_id"],
        "split"           : "train" if i < len(train_samples) else "eval",
        "true_category"   : tgt.get("category",    ""),
        "true_aspect"     : tgt.get("aspect",       ""),
        "true_intent"     : str(tgt.get("intent",   "")),
        "true_explanation": tgt.get("explanation",  ""),
        "true_country"    : tgt.get("country",      ""),
        "true_source"     : tgt.get("source",       ""),
        "true_sector"     : tgt.get("sector",       ""),
        "pred_category"   : str(pred.get("category",    "")).strip(),
        "pred_aspect"     : str(pred.get("aspect",       "")).strip(),
        "pred_intent"     : str(pred.get("intent",       "")).strip(),
        "pred_explanation": str(pred.get("explanation",  "")).strip(),
        "pred_country"    : str(pred.get("country",      "")).strip(),
        "pred_source"     : str(pred.get("source",       "")).strip(),
        "pred_sector"     : str(pred.get("sector",       "")).strip(),
    })

    if (i + 1) % 20 == 0:
        print(f"  Inferred {i+1}/{len(all_samples)}")

print(f"✅ Inference done for {len(all_results)} samples")

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 14 ── Accuracy Scoring
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
scorer = rs.RougeScorer(["rougeL"], use_stemmer=True)

def score_split(rows, label="ALL"):
    if not rows:
        return {}

    def acc(t, p):
        return accuracy_score(
            [x.lower().strip() for x in t],
            [x.lower().strip() for x in p]
        )

    rouge_scores = [
        scorer.score(r["true_explanation"], r["pred_explanation"])["rougeL"].fmeasure
        for r in rows
    ]
    avg_rouge = float(np.mean(rouge_scores))

    cat_acc = acc([r["true_category"] for r in rows], [r["pred_category"] for r in rows])
    asp_acc = acc([r["true_aspect"]   for r in rows], [r["pred_aspect"]   for r in rows])
    int_acc = acc([r["true_intent"]   for r in rows], [r["pred_intent"]   for r in rows])
    cty_acc = acc([r["true_country"]  for r in rows], [r["pred_country"]  for r in rows])
    sec_acc = acc([r["true_sector"]   for r in rows], [r["pred_sector"]   for r in rows])
    src_acc = acc([r["true_source"]   for r in rows], [r["pred_source"]   for r in rows])

    overall = (cat_acc*0.25 + asp_acc*0.25 + int_acc*0.15 +
               avg_rouge*0.20 + sec_acc*0.10 + cty_acc*0.03 + src_acc*0.02)

    return {
        "split"             : label,
        "n_samples"         : len(rows),
        "category_acc"      : round(cat_acc   * 100, 2),
        "aspect_acc"        : round(asp_acc   * 100, 2),
        "intent_acc"        : round(int_acc   * 100, 2),
        "country_acc"       : round(cty_acc   * 100, 2),
        "source_acc"        : round(src_acc   * 100, 2),
        "sector_acc"        : round(sec_acc   * 100, 2),
        "explanation_rougeL": round(avg_rouge * 100, 2),
        "OVERALL_SCORE_%"   : round(overall   * 100, 2),
    }


train_rows   = [r for r in all_results if r["split"] == "train"]
eval_rows    = [r for r in all_results if r["split"] == "eval"]
train_scores = score_split(train_rows,  "TRAIN")
eval_scores  = score_split(eval_rows,   "EVAL")
all_scores   = score_split(all_results, "ALL")

def print_scores(s):
    print(f"\n  ── {s['split']} SET  ({s['n_samples']} samples) ──")
    print(f"   Category    Accuracy  : {s['category_acc']:>7.2f} %   (weight 25%)")
    print(f"   Aspect      Accuracy  : {s['aspect_acc']:>7.2f} %   (weight 25%)")
    print(f"   Intent      Accuracy  : {s['intent_acc']:>7.2f} %   (weight 15%)")
    print(f"   Explanation ROUGE-L   : {s['explanation_rougeL']:>7.2f} %   (weight 20%)")
    print(f"   Sector      Accuracy  : {s['sector_acc']:>7.2f} %   (weight 10%)")
    print(f"   Country     Accuracy  : {s['country_acc']:>7.2f} %   (weight  3%)")
    print(f"   Source      Accuracy  : {s['source_acc']:>7.2f} %   (weight  2%)")
    print(f"   {'─'*42}")
    print(f"   ⭐ OVERALL SCORE      : {s['OVERALL_SCORE_%']:>7.2f} %")

print("\n" + "="*65)
print("              📊 ACCURACY REPORT")
print("="*65)
print_scores(train_scores)
print_scores(eval_scores)
print_scores(all_scores)
print("="*65)

# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# CELL 15 ── Save Results to Excel
# ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
results_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame([train_scores, eval_scores, all_scores])

os.makedirs(os.path.dirname(RESULTS_EXCEL_DRIVE), exist_ok=True)

with pd.ExcelWriter(RESULTS_EXCEL_DRIVE, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Accuracy Summary", index=False)
    results_df.to_excel(writer, sheet_name="Per Sample",       index=False)

    for field in ["category", "aspect", "intent", "sector", "country", "source"]:
        breakdown = results_df[["uni_id","split",f"true_{field}",f"pred_{field}"]].copy()
        breakdown["correct"] = (
            breakdown[f"true_{field}"].str.lower().str.strip() ==
            breakdown[f"pred_{field}"].str.lower().str.strip()
        )
        breakdown.to_excel(writer, sheet_name=f"{field.capitalize()} Detail", index=False)

print(f"\n✅ Results saved → {RESULTS_EXCEL_DRIVE}")
print("\n🎉 ALL DONE!")